In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
from qutip import *
from scipy import *
from numpy import *
from scipy.optimize import minimize
import random
from math import log

from IPython.display import display

from sympy.interactive import printing
printing.init_printing()

from collections import defaultdict

from __future__ import division

from random import randrange
from datetime import datetime

from tempfile import TemporaryFile
arglikelihood = TemporaryFile()
argFid = TemporaryFile()

In [12]:
sx=sigmax()
sy=sigmay()
sz=sigmaz()

xxi = tensor(sx,sx,qeye(2))
yyi = tensor(sy,sy,qeye(2))
zzi = tensor(sz,sz,qeye(2))
zii = tensor(sz,qeye(2),qeye(2))

ixx = tensor(qeye(2),sx,sx)
iyy = tensor(qeye(2),sy,sy)
izz = tensor(qeye(2),sz,sz)
izi = tensor(qeye(2),sz,qeye(2))
HO=(xxi+yyi+zzi+zii)*(ixx+iyy+izz+iiz) - (ixx+iyy+izz+iiz)*(xxi+yyi+zzi+zii)
HOc=HO.conj()
(HO*HOc).tr()

In [3]:
j = complex(0,1)

class PAULI (object) :
    
    name = ''
    op = Qobj()
    
    def __init__(self, name, PauliMatrix) :
        self.name = name
        self.op = PauliMatrix
    
def make_pauli_matrix (name, PauliMatrix) :
    Pmatrix = PAULI (name, PauliMatrix)
    return Pmatrix

 
sx = make_pauli_matrix('X', sigmax())
sy = make_pauli_matrix('Y', sigmay())
sz = make_pauli_matrix('Z', sigmaz())
I = make_pauli_matrix('I', qeye(2))

pauliMatr = [[sz,sz],[sx,sx],[I,sx],[sz,I]]


######################################################
#GATE and DIMENSION
####################################################
N = 4        # n of qubits

G = toffoli()

#interactions = [[0,1],[0,2],[1,2],[0,3],[1,3],[2,3],[0,4],[2,4],[1,4]]

#######################################################################
#USEFUL DEFINITIONS
#######################################################################

dimG = G.shape[0]  
CareStateDim = int(log(dimG,2))

h = N-CareStateDim

b = sin(0.823937)
a = cos(0.823937)

if h > 1 :
    dontCareStates = [b*basis(2,1) + a*basis(2,0)]*(h)
    dontCareIdentity = [qeye(2)]*(h)
else :
    dontCareStates = [b*basis(2,1) + a*basis(2,0)]
    dontCareIdentity = [qeye(2)]


dCS = tensor(dontCareStates)
dCI = tensor(dontCareIdentity)

###########################################################################
#FUNCTIONS
###########################################################################
from pygame import mixer
mixer.init() #you must initialize the mixer
alert = mixer.Sound('small-bell-ringing-02.wav') 

def Likelihood(J, rho_0) : #Likelihood function
    
    Ak = G*rho_0
    #A = tensor(Ak*Ak.dag(), dCI)
    A = Ak*Ak.dag()
    
    rho = tensor(rho_0, dCS)
    H = HamiltonianAB(J)
    U = (-j*H).expm()
    Btemp = U*rho
    Bk = Btemp*Btemp.dag()
    B = Bk.ptrace([0,1,2])
    
    out = (A*B).tr()

    return abs(out)


def Hamiltonian(x) :
    
    k = 0
    H = 0


    for q in [sx,sz]:    
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[0] = q.op
        OpChain[1] = q.op
        temp += x[k]*tensor(OpChain)
        k+=1        
        H += temp 

    for p in [2,3,4]:
        for q in [sx,sz]:
            
            temp = 0
            OpChain = [qeye(2)]*N
            OpChain[0] = q.op
            OpChain[p] = q.op
            
            temp += x[k]*tensor(OpChain)
            
            OpChain = [qeye(2)]*N
            OpChain[1] = q.op
            OpChain[p] = q.op
            
            temp += x[k]*tensor(OpChain)
            k += 1
            
            H += temp 
                
    for p in [3,4]:
        for q in [sx,sz]:

    
            temp = 0
            OpChain = [qeye(2)]*N
            OpChain[2] = q.op
            OpChain[p] = q.op
            temp += x[k]*tensor(OpChain)
            k+=1        
            H += temp 
    

    for i in range(2) :
    
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[i] = sz.op
        temp += x[k]*tensor(OpChain)
    
        H += temp 
        
    k+=1
    temp = 0
    OpChain = [qeye(2)]*N
    OpChain[2] = sz.op
    temp += x[k]*tensor(OpChain)
    
    H += temp 

    for i in range(3) :
        k += 1
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[i+2] = sx.op
        temp += x[k]*tensor(OpChain)#last one

        H += temp 

    
    return H

def HamiltonianAB(x) :
    
    k = 0
    H = 0


    for q in [sx,sz]:    
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[0] = q.op
        OpChain[1] = q.op
        temp += x[k]*tensor(OpChain)
        k+=1        
        H += temp 

    for p in [2,3]:
        for q in [sx,sz]:
            
            temp = 0
            OpChain = [qeye(2)]*N
            OpChain[0] = q.op
            OpChain[p] = q.op
            
            temp += x[k]*tensor(OpChain)
            
            OpChain = [qeye(2)]*N
            OpChain[1] = q.op
            OpChain[p] = q.op
            
            temp += x[k]*tensor(OpChain)
            k += 1
            
            H += temp 
            
   
    for q in [sx,sz]:
        
            
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[2] = q.op
        OpChain[3] = q.op
        temp += x[k]*tensor(OpChain)
        k+=1    
        H += temp 
            
            
    for i in range(2) :
        
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[i] = sz.op
        temp += x[k]*tensor(OpChain)
        H += temp 

    k += 1

    for i in [2,3] :
    
        temp = 0
        OpChain = [qeye(2)]*N
        OpChain[i] = sx.op
        temp += x[k]*tensor(OpChain)
        k += 1    

        OpChain = [qeye(2)]*N
        OpChain[i] = sz.op
        temp += x[k]*tensor(OpChain)#last one
        k += 1

        H += temp 

    
    return H


In [4]:
def getGate (G): #extract non zero elements of the gate and save them in s
    
    s = []

    rows = G.shape[0]
    colums = G.shape[1]
    
    for i in range(rows):
        for j in range(colums):
            if G[i][0][j] != 0 :
                s.append([i,j])
    return s

def bin(a):  #binary representation of a number a: useful to write the computational basis 
    s=''                                          #inside the Fidelity
    t={'0':'000','1':'001','2':'010','3':'011',
       '4':'100','5':'101','6':'110','7':'111'}
    for c in oct(a)[1:]:
            s+=t[c]
    return s

def getBasis (a) : #get the basis states according to the binary of a: 10 -> |10>
    
    if a == 0:
          B = [basis(2,0)]*(CareStateDim)
          return tensor(B)
    
    c = bin(a)
    if dimG == 8:
        return tensor(basis(2,int(c[0])) , basis(2,int(c[1])), basis(2,int(c[2])))
    if dimG == 4:
        return tensor(basis(2,int(c[1])) , basis(2,int(c[2])))
    
def Fidelity (J):  #Fidelity function
    
    s = getGate(G)
    H = HamiltonianAB(J)
    Fid = 1/(dimG + 1)
    U = (-j*H).expm()
    Udag = (j*H).expm()
    
    for x in s :
        for y in s:
            
            #definition of the basis kets and bras.             
            bra_i = getBasis(x[0]).dag()
            ket_j = getBasis(y[0])
            ket_k = getBasis(x[1])
            bra_l = getBasis(y[1]).dag()
            
            Epsilon = U*tensor(ket_k*bra_l, dCS*dCS.dag())*Udag
            Eps_ijkl = bra_i*(Epsilon.ptrace([0,1,2]))*ket_j
            
            Gstar_ik = G[x[0],x[1]].conjugate()
            G_jl = G[y[0],y[1]]
            
            
            fidStep = (1/(dimG*(dimG+1)))*Gstar_ik*Eps_ijkl*G_jl     
            Fid += fidStep[0][0][0]
            
    return -abs(Fid)

def LocGradient(J,s) :
    
    dL = []
    delta = 0.0001
    for h in range(len(J)) :
        JdJ = [x for x in J]
        JdJ[h] += delta
        FdJ = Fidelity(JdJ)
        dL.append((FdJ - s)/delta)
    
    return dL, FdJ

def GradDesc(Jopt) :
    
    J = [x for x in Jopt]
    check = 0
    s = 1
    i = 16
    g = 0
    s = Fidelity(J)
    print(s)
    while True :   
        
        g = s
        grad, s = LocGradient(J,s)
        J = [J[k] - grad[k]/sqrt(i) for k in range(len(J))]
    
        s = Fidelity(J) 
        t.write(str(i)+'    '+str(s)+'\n')
        

        print(s)
        
        if abs(s-g) < 0.00005 :
            check += 1    
            if check == 20:
                break
        
        i += 0.5
       
    return J



In [5]:
Jopt=zeros(13)
Jopt[6] = 15.1043/4
Jopt[5] = -5.66207/4
Jopt[1] = -8.95113/4
Jopt[3] = -4.9753/4
Jopt[9] = -19.0948/2
Jopt[11] = -4.30184/2
Jopt[8] = -2.43/2
Jopt[10] = -4.98238/2
Jopt[12] =-0.208699/2
Fidelity(Jopt)

In [11]:
ide = array([1,0,0,1])
idet = ide.T
idet*ide

array([1, 0, 0, 1])

In [89]:
Jopt+pi

array([ 3.14159265,  0.90381015,  3.14159265,  1.89776765,  3.14159265,
        1.72607515,  6.91766765,  3.14159265,  1.92659265, -6.40580735,
        0.65040265,  0.99067265,  3.03724315])

In [11]:
Jgrad = Jopt*180/pi
Jgrad

array([   0.        , -128.21549272,    0.        ,  -71.26592295,
          0.        ,  -81.10317858,  216.35316062,    0.        ,
        -69.61437211, -547.02572532, -142.73467297, -123.23863807,
         -5.97878594])

In [94]:
Jgrad = Jopt*180/pi
Jgrad[1] = -128
Jgrad[3] = -70
Jgrad[5] = -81
Jgrad[6] = 216
Jgrad[8] = -70
Jgrad[9] = -547
Jgrad[10] = -143
Jgrad[11] = -123
Jgrad[12] = -6
k = Jgrad*pi/180
k = k + 4*pi
k[0] = 0
k[2] = 0
k[4] =0
k[7] = 0
Fidelity(k)

In [78]:
547/2

In [85]:
for p in range(2,274):
    if 123%p == 0:
        print p
        
        

3
41
123


In [7]:
t = open('nonStochasticToffoli', 'w+')
Jini = Jopt
startTime = datetime.now()
J2 = GradDesc(Jini)
print(datetime.now()-startTime)
print(J2)
t.close()

-0.94376739569
-0.951185797356
-0.953449815795
-0.954956510758
-0.956061851064


KeyboardInterrupt: 

In [51]:
res_minimize

  status: 1
    nfev: 7200
 success: False
     fun: -0.99985694688255611
       x: array([  3.01244967e-02,   3.72743088e-04,   2.57327936e+00,
         2.62808777e-01,  -2.83541700e-03,   9.31211181e-01,
        -1.58564974e-01,   5.30525197e+00,   1.19616443e-02,
         2.57801583e+00,   1.67278970e-01,   9.57916862e-01,
         2.20053445e-02,   9.18940162e-01,   1.58409399e+00,
        -4.93167266e+00,   1.20311415e-03,   7.52463461e-03,
         1.35185346e+00,  -4.16403595e+00,   2.19526493e-02,
         1.26387633e+00,   5.82067710e-01,   1.91335379e-02,
         1.47539488e-02,   5.73897488e-02,   1.64147396e+00,
         1.86562320e+00,   1.23420311e-03,   2.08551303e-01,
         7.45019117e-01,  -2.82141305e-02,  -1.64538808e+00,
         2.63695234e-02,   4.96263673e-01,   3.84521672e-02])
 message: 'Maximum number of function evaluations has been exceeded.'
     nit: 6052

In [6]:
J2 = [2.0565948620691823, 0.48226382405442797, 0.15393134664380062, -0.33289288739898715, 0.82714531675280167, 0.010563507543686479, 0.82106896438319665, 4.2165802897113487, 0.83343110003058118, -0.010683906294527919, 3.8209462625416766, 1.8665218299625692, 1.5043852083566869, -0.22816425207220267, -0.085767107039843249, 1.6886455914722138, 1.5325559085730567, -0.070158653304250168, -0.23117237447353578, 1.1518163641576706, -0.13880470709459947, 1.1285130855256897, 0.62771587889985092, 4.288834120929649, 0.01522806647698545, 0.24187071928102979, 1.0601273123008075, 0.33610918013820928, 0.014902613959011206, -0.00026858745357400462, 0.78766220308788482, -1.8008963132270512, 0.0053091449578853483, -0.14276279448268453, 0.75052392710876359, -0.19887759260445298]

In [7]:
from decimal import *
getcontext().prec = 5
i = s = 0
N = 5
Jopt = J2
for f in pauliMatr :
    s += 1
    c = 'J_' + f[0].name + f[1].name
    J = zeros(shape=(N,N))
    for k in interactions :
        a = "{0:.3f}".format(Jopt[i])
        val = float(a)
        J[k[0],k[1]] = val
        i += len(pauliMatr)
    i = s
    exec(c + '= asmatrix(J)')

In [8]:
J_ZI

matrix([[ 0.   , -0.333,  4.217,  1.689,  0.336],
        [ 0.   ,  0.   ,  1.867,  1.152, -0.199],
        [ 0.   ,  0.   ,  0.   ,  4.289, -1.801],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [13]:
sum(J_ZI, axis=0)

matrix([[ 0.   , -0.333,  6.084,  7.13 , -1.664]])

In [9]:
J_IX

matrix([[ 0.   ,  0.154,  0.821, -0.086,  1.06 ],
        [ 0.   ,  0.   ,  3.821, -0.231,  0.751],
        [ 0.   ,  0.   ,  0.   ,  0.628,  0.788],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [14]:
sum(J_IX, axis=0)

matrix([[ 0.   ,  0.154,  4.642,  0.311,  2.599]])

In [10]:
J_XX

matrix([[ 0.   ,  0.482,  0.011, -0.228,  0.242],
        [ 0.   ,  0.   , -0.011, -0.07 , -0.143],
        [ 0.   ,  0.   ,  0.   ,  1.129, -0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [15]:
sum(J_XX, axis=0)

matrix([[ 0.   ,  0.482,  0.   ,  0.831,  0.099]])

In [11]:
J_ZZ

matrix([[ 0.   ,  2.057,  0.827,  1.504,  0.015],
        [ 0.   ,  0.   ,  0.833,  1.533,  0.005],
        [ 0.   ,  0.   ,  0.   , -0.139,  0.015],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [16]:
sum(J_ZZ, axis=0)

matrix([[ 0.   ,  2.057,  1.66 ,  2.898,  0.035]])